<a href="https://colab.research.google.com/github/FFI-Vietnam/camtrap-tools/blob/main/MegaDetector/confusion_matrix/01_ground_truth_from_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This script creates a groundtruth table which specifies what species inside an 
image from the exiftool metadata file. The species information is extracted 
from **CatalogSets** column, and image names are extracted from **FileName** column
"""

In [12]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# species Colab directories
root = '/content/drive/'
contain_folder = 'My Drive/FFI/MegaDetector Test/'
file_name = 'image_metadata(2020-06-26)_full.csv'

In [17]:
# read ExifTool spreadsheet
file_path = os.path.join(root, contain_folder, file_name)
full_metadata = pd.read_csv(file_path)
full_metadata.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,20,41,42,45,56,57,58,60,61,75) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SourceFile,BitsPerSample,Caption-Abstract,CatalogSets,Categories,CodedCharacterSet,ColorComponents,ColorLabel,Comment,CreateDate,CreatorTool,CurrentIPTCDigest,DateCreated,DateTime,DateTimeCreated,DateTimeOriginal,DateUTC,Description,Directory,EncodingProcess,Error,ExifByteOrder,ExifImageHeight,ExifImageWidth,ExifToolVersion,ExifVersion,FileAccessDate,FileCreateDate,FileModifyDate,FileName,FilePermissions,FileSize,FileType,FileTypeExtension,Flash,HierarchicalSubject,ImageDescription,ImageHeight,ImageNumber,ImageSize,ImageWidth,InstanceID,IPTCDigest,JFIFVersion,Keywords,LastKeywordIPTC,LastKeywordXMP,Make,Megapixels,MetadataDate,MIMEType,Model,ModificationDate,ModifyDate,Notes,OffsetSchema,Orientation,OriginatingProgram,Padding,PickLabel,ProcessingSoftware,ProgramVersion,Rating,RatingPercent,RegionInfoRegions,RegionList,ResolutionUnit,Software,Subject,TagsList,TimeCreated,Urgency,UserComment,Warning,XMPToolkit,XPKeywords,XResolution,YCbCrSubSampling,YResolution
0,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:09:11,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:09:11,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:05:26 10:18:20+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-09-11(1).JPG,rw-rw-rw-,458 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1154.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:09:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
1,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:09:42,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:09:42,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:06:24 14:00:28+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-09-42(2).JPG,rw-rw-rw-,471 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1155.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:09:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
2,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Setup, Observer|Tam","<Categories><Category Assigned=""0"">People<Cate...",UTF8,3.0,NaN,NaN,2019:04:22 15:12:32,NaN,0dfadea9315c091a67a43eb6881ea5df,NaN,NaN,NaN,2019:04:22 15:12:32,NaN,NaN,D:/Fauna & Flora International/VietnamCameraSt...,"Baseline DCT, Huffman coding",NaN,"Big-endian (Motorola, MM)",1536.0,2048.0,11.63,220.0,2020:06:24 14:00:29+07:00,2020:03:20 13:23:52+07:00,2020:05:07 10:40:30+07:00,100__67096__2019-04-22__15-12-32(1).JPG,rw-rw-rw-,485 kB,JPEG,jpg,Fired,"People|Setup, Observer|Tam",integtime=1535 A=88 D=128\r\n\rflashCrossover...,1536.0,1155.0,2048x1536,2048.0,NaN,NaN,1.02,"Setup, Tam",NaN,"People/Setup, Observer/Tam",Panthera V4,3.1,NaN,image/jpeg,CAM67096,NaN,2019:04:22 15:12:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102717 V6-Bld1.15,"Setup, Tam","People/Setup, Observer/Tam",NaN,NaN,NaN,[minor] Unrecognized MakerNotes,XMP Core 4.4.0-Exiv2,NaN,1.0,YCbCr4:2:2 (2 1),1.0
3,D:/Fauna & Flora International/VietnamCameraSt...,8.0,NaN,"People|Se

In [4]:
# keep 'FileName' and 'CatalogSets' columns
metadata = full_metadata[['FileName', 'CatalogSets']]

metadata.head()

,FileName,CatalogSets
0,100__67096__2019-04-22__15-09-11(1).JPG,"People|Setup, Observer|Tam"
1,100__67096__2019-04-22__15-09-42(2).JPG,"People|Setup, Observer|Tam"
2,100__67096__2019-04-22__15-12-32(1).JPG,"People|Setup, Observer|Tam"
3,100__67096__2019-04-22__15-12-55(2).JPG,"People|Setup, Observer|Tam"
4,100__67096__2019-04-22__15-13-18(1).JPG,"People|Setup, Observer|Tam"


In [7]:
# split to species_common_name column, each row represents at most one species 
# of each image. If one image has more than one species, it will be exploded into
# multiple entris

species_common_name = [] 

# prefix in CatalogSets column
SPECIES_PREFIX = 'Species'
PEOPLE_PREFIX = 'People'

catalog_sets = metadata['CatalogSets']
for i in range(len(catalog_sets)):
  if catalog_sets[i] is not np.NaN:
    catalog = str(catalog_sets[i]).split(', ')
    spe = ''
    for i in range(len(catalog)):
      if SPECIES_PREFIX in catalog[i]:
        if catalog[i].split('|')[1] != 'Human':
          if '(unknown)' in catalog[i].split('|')[1]: # remove 'unknown' mark
            spe = spe + ', ' + catalog[i].split('|')[1].split('(unknown)')[0]
          else:
            spe = spe + ', ' + catalog[i].split('|')[1]
      # Human is a bit tricky, it either has 'People' or 'Human' prefix, or both, 
      # so have to check to avoid duplication
      if PEOPLE_PREFIX in catalog[i]:
        if not 'Human' in spe:
          spe = spe + ', ' + 'Human'
    species_common_name.append(spe[2:])

  else:
    species_common_name.append('')

metadata['species_common_name'] = species_common_name

col = metadata.columns.to_list()
col.remove('species_common_name')

# explode an image which has multiple species into multiple rows
metadata = metadata.set_index(col).apply(lambda x: x.str.split(',').explode()).reset_index()
metadata['species_common_name'] = metadata['species_common_name'].str.strip()

metadata.sample(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,FileName,CatalogSets,species_common_name
11309,179__67111__2019-06-17__05-08-51(3).JPG,NaN,
27452,237__68840__2019-05-31__21-10-09(1).JPG,NaN,
110225,99__67135__2019-06-28__20-04-56(1).JPG,NaN,
28927,kpc239__ct68975__2019-10-09__17-17-40(1)__Slat...,"Species|Slaty-legged Crake, Observer|An Nguyen",Slaty-legged Crake
49680,280__67163__2019-12-10__16-25-22(1).JPG,Observer|Tam,
19980,kpc222__ct68861__2019-11-18__01-18-07(1)__Blan...,Observer|An Nguyen,
31011,kpc239__ct69010__2019-11-17__13-23-39(3)__Blan...,Observer|An Nguyen,
108578,96__66983__2019-07-05__00-11-33(2).JPG,NaN,
59856,311__69025__2019-06-09__01-02-48(1).JPG,NaN,
11301,179__67111__2019-06-17__04-56-08(1).JPG,NaN,


In [52]:
# save file to Colab runtime storage (will be deleted when this notebook is closed)
metadata.to_csv('metadata.csv', index=False)

# save file back to Google Drive for permanent storage
ground_truth_folder = 'ground_truths/' # create ground_truths folder
ground_truth_file = 'ground-truth-table_Kon-Plong.csv'
ground_truth_path = os.path.join(root, contain_folder, ground_truth_folder, ground_truth_file)
try:
  os.makedirs(ground_truth_folder)
except:
  pass

with open('metadata.csv', 'r') as f:
  metadata_file = f.read()

with open(ground_truth_path, 'w') as f:
  f.write(metadata_file)

# sanity check, print "Error while saving" if something wrong
ground_truth_df = pd.read_csv(ground_truth_path).fillna('')
metadata = metadata.fillna('')
assert ground_truth_df.equals(metadata) , "Error while saving"

In [11]:
print(f"# animal images / # images = {len(metadata[metadata['species_common_name'] != ''])/len(metadata)}")

# animal images / # images = 0.24818252687684694
